In [6]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
tf.test.is_built_with_cuda()
import os, sys
sys.path.append('../')
import torch
import torch.nn as nn 
import math
from torch import nn, Tensor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from save_best_model import SaveBestModel
from sandpile import Sandpile, run_sandpile_alone
import random
from collections import deque
from torch.distributions import Categorical
import time
import datetime
from rl_agents import Policy

# Set the seed value all over the place to make this reproducible.
seed_val = 42


random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)


#RUN THIS ON COLAB
ON_COLAB = False
if ON_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    drive_path = '/content/drive/MyDrive/Phase ML Data/'

In [8]:
import os

# Saving best-practices: if you use defaults names for the model, you can reload it using from_pretrained()
model_nickname = 'reinforce-agent'

output_dir = f'/staging_area/{model_nickname}/'

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

checkpoint_dir = 'checkpoints/'
if not os.path.exists(output_dir+checkpoint_dir):
    os.makedirs(output_dir+checkpoint_dir)


best_model_name = 'best_agent.tar'
save_best_model = SaveBestModel(output_dir+best_model_name)

Saving model to /staging_area/reinforce-agent/


In [ ]:

def enum_parameters(model):
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        total_params+=params
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
# enum_parameters(model)

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")


if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.998), eps=1e-9, weight_decay=1e-4)

start_epoch = 0
FROM_CHECKPOINT = not True
if FROM_CHECKPOINT:
    
    checkpoint = torch.load(output_dir+'best_gait_model.tar')
    g = checkpoint['model_state_dict']
    loss = checkpoint['loss']
    print(f'Lowest Loss: {loss}')
    save_best_model = SaveBestModel(output_dir+best_model_name, loss)
    # print(g.keys())
    model.load_state_dict(g)

    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch']
    print(f'From epoch: {start_epoch}')



epochs = 5
SAVE_EVERY_EPOCH_N = 1
    
#define loss function
lossfcn = nn.MSELoss

training_RMSEs = []
validation_RMSEs = []



# Measure the total training time for the whole run.
total_t0 = time.time()

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))



for epoch_i in range(start_epoch, start_epoch+epochs):
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, start_epoch+epochs))
    print('Training...')
    
    # lr = scheduler.get_lr()
    lr = optimizer.param_groups[0]['lr']
    print(f'Learning Rate: {lr}')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    model.train()
    step_ct = 0
    for step, batch in enumerate(train_dataloader):
        step_ct += 1
        # for step, batch in enumerate([first_batch] * 200):

        
        optimizer.zero_grad()   
        
        b_kinematics = batch['meas'].to(device)
        b_state = batch['state'].to(device)
        
    
        #Run the kinematics model
        #extract only the last (most recent temporally) element of the kinematics
        #also don't estimate time
        b_kinematics = b_kinematics[:,-1,:-1].unsqueeze(dim=1)
        # print(b_kinematics.shape)
        tgt = SOS_token.repeat(b_state.shape[0], 1, 1)

        # predicted_kinematics = model(b_state,tgt)
        predicted_kinematics = model(b_state)
        
        loss = lossfcn(predicted_kinematics, b_kinematics,b_state)
        # print(loss)

        loss.backward()

        optimizer.step()

        loss_value = loss.item()

        # Progress update every 40 batches.
        if step % 100 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            print(loss_value)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss_value
        
    # Update the learning rate.
    #if StepLR
    # scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / step_ct           
    training_RMSEs.append(avg_train_loss)
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.4f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
    
    # Save to checkpoints
    if (epoch_i + 1) % SAVE_EVERY_EPOCH_N == 0:
        print('Saving model checkpoint')
        model_name = f"gait_model_checkpoint_{epoch_i + 1}.tar"
        path_name = output_dir+checkpoint_dir+model_name
        print(path_name)
        torch.save({
                    'epoch':epoch_i + 1,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': loss.item(),
                    }, path_name)

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    
    
    step_ct = 0
    for batch in validation_dataloader:
    #for batch in train_dataloader:
    # for step, batch in enumerate([first_batch] * 10):
        step_ct += 1

        b_kinematics = batch['meas'].to(device)
        b_state = batch['state'].to(device)
        
        
        tgt = SOS_token.repeat(b_state.shape[0], 1, 1)
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            
            #Run the gait model model
            #extract only the last (most recent temporally) element of the kinematics
            #also don't predict time step
            b_kinematics = b_kinematics[:,-1,:-1].unsqueeze(dim=1)
            
            outputs = model(b_state)
            # outputs = model(b_state,tgt)

        loss = lossfcn(outputs, b_kinematics,b_state)
        #print(loss)
        loss_value = loss.item()
        #print(loss_rmse)

        

        # Accumulate the validation loss.
        total_eval_loss += loss_value


    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / step_ct
    
    
    validation_RMSEs.append(avg_val_loss)

    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.4f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))
    
    #save best model
    save_best_model(
        avg_val_loss, epoch_i+1, model, optimizer, lossfcn
    )
    
    #print training vals
    print('Training vals')
    print(validation_RMSEs)

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

print("Saving model to %s" % output_dir)

#save model params
model_name = 'gait_model_params.pt'
torch.save(model.state_dict(), output_dir+model_name)

model_name = 'gait_model_full.pt'
torch.save(model, output_dir+model_name)

#save checkpoint
model_name = f"gait_model_checkpoint_{epoch_i + 1}.tar"
torch.save({'epoch': epoch_i + 1,
                      'model_state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict(),
                      'loss': loss.item(),
                      }, path_name)

In [ ]:
training_RMSEs = np.array(training_RMSEs)
validation_RMSEs = np.array(validation_RMSEs)
fig, axs = plt.subplots()
axs.plot(training_RMSEs,'-',label='Train')
axs.set_ylabel('Loss (MSE)')
axs.plot(validation_RMSEs,'-',label='Val')
axs.set_xlabel('Epoch')
axs.legend()
print(np.min(validation_RMSEs))

In [ ]:
#Test model

if REMOVE_SUBS_XVAL:
    test_dataset = val_dataset
else:
    # Create the DataLoader.
    filename_r01_ordered = 'r01_ordered_corrupt_time.csv'
    filename_r01_randomized = 'r01_randomized_corrupt_time.csv'
    filename_dataport_ordered = 'dataport_ordered_corrupt_time.csv'
    filename_dataport_randomized = 'dataport_randomized_corrupt_time.csv'

    wgd_small = pd.read_csv(filename_r01_randomized,skiprows=0,nrows=10000)
    # wgd_small = pd.read_csv(filename_dataport_randomized,skiprows=0,nrows=10000)
    # wgd_small = pd.read_csv(filename_r01_randomized,skiprows=0)
    window_size = 100
    meas_scale = np.array([[-69.35951035,  27.62815047],\
                            [-456.18013759,  401.13782617],\
                            [-63.71649984,  22.06632622],\
                            [-213.4786175,   396.93801619],\
                            [-35.26603985,  20.78473636],\
                            [-20.95456523,  14.63961137],\
                              [0,1]])
    
    
    speed_scale = (0,2)
    incline_scale = (-10,10)
    do_phase_trig = True



    test_dataset = WindowedGaitDataset(gait_data=wgd_small,
                                                window_size = window_size,
                                                meas_scale = meas_scale,
                                                speed_scale = speed_scale,
                                                incline_scale = incline_scale,
                                                transform=ToTensor())   

#test_dataset = train_dataset
BATCH_SIZE = 1024*4
prediction_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,shuffle=False,num_workers=8)
# Prediction on test set

print('Predicting labels for {:,} test points...'.format(len(test_dataset)))


# SET UP KINEMATICS MODEL
dim_val = 64 # 
n_hidden_layers=4
input_size = 6 # The number of input variables. 1 if univariate forecasting.
num_predicted_features = 6 # The number of output variables. 

best_model = GaitModel(
    input_size=input_size,
    num_predicted_features=num_predicted_features,
    dim_val=dim_val,  
    n_hidden_layers=n_hidden_layers
)

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")


if torch.cuda.device_count() > 1:
    print("Using", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    model = nn.DataParallel(model)
best_model.to(device)


model_nickname = 'gait-model'

model_dir = f'../staging_area/{model_nickname}/model_save/'
if REMOVE_SUBS_XVAL:
    model_dir = f'../staging_area/{model_nickname}/model_save_xval/'

checkpoint = torch.load(model_dir+'best_gait_model.tar')
g = checkpoint['model_state_dict']
loss = checkpoint['loss']
print(f'Lowest Loss: {loss}')
best_model.load_state_dict(g)

epoch = checkpoint['epoch']

# Put model in evaluation mode
best_model.eval()


#Set up start of sequence token
SOS_token = 100 * torch.ones(1, 1, num_predicted_features).to(device).requires_grad_(False)

# Tracking variables 
predictions , true_labels = [], []

state_vars = []

# Predict 
for batch in prediction_dataloader:

    b_kinematics = batch['meas'].to(device)
    b_state = batch['state'].to(device)
    tgt = SOS_token.repeat(b_state.shape[0], 1, 1)
    
    # Telling the model not to compute or store gradients, saving memory and 
    # speeding up prediction
    with torch.no_grad():
        
        #Run the gait model
        #extract only the last (most recent temporally) element of the kinematics
        b_kinematics = b_kinematics[:,-1,:-1]
        # outputs = best_model(b_state,tgt)
        outputs = best_model(b_state)
        # print(outputs)

    # Move logits and labels to CPU
    outputs = outputs.detach().to('cpu').numpy()
    b_kinematics = b_kinematics.to('cpu').numpy()
    b_state = b_state.to('cpu').numpy()
  
    # Store predictions and true labels
    outputs = np.squeeze(outputs, axis=1)
    b_state = np.squeeze(b_state, axis=1)
    
    # print(outputs.shape)
    
    #unscale
    outputs = unscale_kinematics(outputs, meas_scale)
    b_kinematics = unscale_kinematics(b_kinematics, meas_scale)
    b_state = unscale_gait_state(b_state, speed_scale, incline_scale, stair_height_scale)

    # Store predictions and true labels
    predictions.extend(outputs.tolist())
    true_labels.extend(b_kinematics.tolist())
    state_vars.extend(b_state.tolist())
print('    DONE PREDICTING')

In [ ]:
predictions = np.array(predictions)
true_labels = np.array(true_labels)
state_vars = np.array(state_vars)

#bound the accels at +- 100 to filter out the occasional spikes 
predictions[:,4] = np.clip(predictions[:,4],-100,100)
predictions[:,5] = np.clip(predictions[:,5],-100,100)

true_labels[:,4] = np.clip(true_labels[:,4],-100,100)
true_labels[:,5] = np.clip(true_labels[:,5],-100,100)



print(predictions.shape)
print(true_labels.shape)

foot_angle_losses = np.sqrt((predictions[:,0] - true_labels[:,0])**2)
foot_angle_vel_losses = np.sqrt((predictions[:,1] - true_labels[:,1])**2)
shank_angle_losses = np.sqrt((predictions[:,2] - true_labels[:,2])**2)
shank_angle_vel_losses = np.sqrt((predictions[:,3] - true_labels[:,3])**2)
heel_acc_forward_losses = np.sqrt((predictions[:,4] - true_labels[:,4])**2)
heel_acc_up_losses = np.sqrt((predictions[:,5] - true_labels[:,5])**2)


print("="*30)
print(f'Foot Angle Losses: {np.mean(foot_angle_losses):.3f} +- {np.std(foot_angle_losses):.3f}')
print(f'Foot Angle Vel Losses: {np.mean(foot_angle_vel_losses):.3f} +- {np.std(foot_angle_vel_losses):.3f}')
print(f'Shank Angle Losses: {np.mean(shank_angle_losses):.3f} +- {np.std(shank_angle_losses):.3f}')
print(f'Shank Angle Vel Losses: {np.mean(shank_angle_vel_losses):.3f} +- {np.std(shank_angle_vel_losses):.3f}')
print(f'Heel Acc Forward Losses: {np.mean(heel_acc_forward_losses):.3f} +- {np.std(heel_acc_forward_losses):.3f}')
print(f'Heel Acc Up Losses: {np.mean(heel_acc_up_losses):.3f} +- {np.std(heel_acc_up_losses):.3f}')



fig, axs = plt.subplots(6,1,figsize=(20,16),sharex=True)
axs[0].plot(true_labels[:,0],'b',label='actual')
axs[0].plot(predictions[:,0],'r',label='predict')
axs[0].plot(state_vars[:,0]*30,'k',label='phase')
axs[0].plot(state_vars[:,3]*50,'g',label='is_stairs')

axs[0].legend()
# axs[0].set_xlim([9000,20000])
axs[0].set_xlim([0.3622e6,0.3725e6]) #r01 randomized mix of stairs and inclines
# axs[0].set_xlim([2e6,2.0025e6]) #dataport mix of ramps
# axs[0].set_xlim([3.504e6,3.508e6]) #ordered gt

# axs[0].set_xlim([0.51575e6,0.516e6])

axs[0].set_ylabel('Foot Angle')

axs[1].plot(true_labels[:,1],'b',label='actual')
axs[1].plot(predictions[:,1],'r',label='predict')
axs[1].set_ylabel('Foot Angle Vel')
axs[3].set_ylim([-500,600])

axs[2].plot(true_labels[:,2],'b',label='actual')
axs[2].plot(predictions[:,2],'r',label='predict')
axs[2].set_ylabel('Shank Angle')

axs[3].plot(true_labels[:,3],'b',label='actual')
axs[3].plot(predictions[:,3],'r',label='predict')
axs[3].set_ylabel('Shank Angle Vel')
axs[3].set_ylim([-1000,1000])

axs[4].plot(true_labels[:,4],'b',label='actual')
axs[4].plot(predictions[:,4],'r',label='predict')
axs[4].plot(state_vars[:,0]*10,'k',label='phase')
axs[4].plot(state_vars[:,3]*30,'g',label='is_stairs')

axs[4].set_ylabel('Heel Acc Forward')
axs[4].set_ylim([-50,50])

axs[5].plot(true_labels[:,5],'b',label='actual')
axs[5].plot(predictions[:,5],'r',label='predict')
axs[5].set_ylabel('Heel Acc Up')
axs[5].set_ylim([-50,50])
plt.show()


In [ ]:
params = list(best_model.named_parameters())
print('The model has {:} different named parameters.\n'.format(len(params)))
for p in params:
    # print('p')
    # print(p[0])
    # print(p[1].data)
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

from prettytable import PrettyTable

def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params
    
count_parameters(best_model)


In [ ]:
!gsutil cp -r ../staging_area/gait-model ../full_models/
!zip -r ../full_models/gait-model.zip ../full_models/gait-model/

In [ ]:
!gsutil cp -r ../full_models/gait-model/ gs://ml_gait_estimation/full_models/
!gsutil cp ../full_models/gait-model.zip gs://ml_gait_estimation/full_models/
